In [ ]:
import xarray as xr
import numpy as np

In [3]:
import xarray as xr
import numpy as np
import pandas as pd
import sys
from datetime import datetime
# might be a problem for ML data, since the data is quite large in size
# Define a function to open datasets and concatenate them
def open_and_concatenate(year, variable, months, way, level=0):
    datasets = []
    for month in months:
        dataset = xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc')
        if variable == 'geopotential' and level != 0:
            dataset = dataset.sel(level=level)
        
        # Create a date range with 3-hour intervals starting from midnight
        start = pd.Timestamp(f"{year}-{month}-01 00:00:00")
        if month == 12:
            end = pd.date_range(start=f"{year}-{month}-01", end=f"{str(int(year)+1)}-01-01", freq='M')[0] + pd.Timedelta(hours=21)
        else:
            end = pd.date_range(start=f"{year}-{month}-01", end=f"{year}-{month+1}-01", freq='M')[0] + pd.Timedelta(hours=21)
        date_range = pd.date_range(start, end, freq='3H')

        # Select the data at the specific timesteps
        dataset = dataset.sel(time=date_range)
        
        datasets.append(dataset)
        dataset.close()

    return xr.concat(datasets, dim='time')

# Define a function to calculate statistics
def calculate_statistics(data_array):
    return {
        'mean': np.mean(data_array),
        'min': np.min(data_array),
        'max': np.max(data_array),
        'std': np.std(data_array),
    }

# Function to log processing details
def log_processing(variable, year, level, storm_number):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_message = f'Processed variable: {variable}, Year: {year}, Level: {level}, Timestamp: {timestamp}, Storm number:{storm_number}'
    with open(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets_3h/processing_log_3h.txt', 'a') as log_file:
        log_file.write(log_message + '\n')

# Main function to process data
def process_data(variable, year, level=0):
    year = int(year)
    year_next = year + 1
    month_act = [10, 11, 12]
    month_next = [1, 2, 3]
    if variable == 'geopotential':
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5_hourly_PL/'
    else:
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

    # Open and concatenate datasets
    if year == 1990:
        dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
        dataset = dataset.chunk({'time': 10})
    elif year == 2021:
        dataset = open_and_concatenate(str(year), variable, month_next, way, level)
    else:
        dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
        dataset = dataset.chunk({'time': 10})

    # Determine the specific variable to extract
    specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time', 'level'])

    # Import all tracks and convert dates
    dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
    dates['year'] = dates['start_date'].dt.year

    # Find the indices for storms within the specified timeframe
    if year == 1990:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
    elif year == 2021:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
    else:
    # Chercher start_october dans year, sinon chercher dès janvier de year_next
        index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month >= 1))].index[0]
        index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
        #print(index_start_october, index_end_march_first, '3rd condition start_october + index_end_march_first')
        if len(index_end_march_first) > 0:
            index_end_march = index_end_march_first[-1]
            #print(index_end_march, 'index_end_march 1st condition of 2nd condition')
        else:
            # Si year_next ne renvoie rien, chercher la dernière instance de tempête dans year
            index_end_march = dates[((dates['end_date'].dt.year == year) & (dates['end_date'].dt.month <= 12))].index[-1]
            #print(index_end_march, 'index_end_march 2nd condition of 2nd condition')
    # Process each storm
    for i in range(index_start_october, index_end_march + 1):
        track = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/tc_irad_tracks/tc_3_hours/tc_irad_{i+1}.txt')
        start_date = dates.at[i, 'start_date']
        end_date = dates.at[i, 'end_date']
        storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

        # Initialize lists to store statistics
        stats = {'mean': [], 'min': [], 'max': [], 'std': []}
        #, 'skewness': [], 'kurtosis': []

        # Calculate statistics for each time step
        for time_step in storm_data.time:
            data_slice = storm_data.sel(time=time_step).values
            step_stats = calculate_statistics(data_slice)
            for key in stats:
                stats[key].append(step_stats[key])

        # Save statistics to CSV files
        for key in stats:
            pd.DataFrame(stats[key]).to_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets_3h/{variable}/storm_{i+1}/{key}_{i+1}_{level}.csv')

    # Log the processing details
    log_processing(variable, year, level, i+1)

In [ ]:
variable = 'geopotential'
year = sys.argv[2]
level = sys.argv[3]
process_data(variable, year, level)

In [4]:
year = 2005
variable = 'geopotential'
months = [1, 2, 3]
way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5_hourly_PL/'
level = 1000

In [5]:
test_open = open_and_concatenate(year, variable, months, way, level)

In [6]:
test_open

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 720)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    level      int32 1000
  * time       (time) datetime64[ns] 2005-01-01 ... 2005-03-31T21:00:00
Data variables:
    z          (time, latitude, longitude) float32 1.204e+03 1.204e+03 ... 271.1
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-12 04:34:52 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [66]:
datasets = []

for month in months:
    dataset = xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc')
    if variable == 'geopotential' and level != 0:
        dataset = dataset.sel(level=level)
    
    # Create a date range with 3-hour intervals starting from midnight
    start = pd.Timestamp(f"{year}-{month}-01 00:00:00")
    if month == 12:
        end = pd.date_range(start=f"{year}-{month}-01", end=f"{str(int(year)+1)}-01-01", freq='M')[0] + pd.Timedelta(hours=21)
    else:
        end = pd.date_range(start=f"{year}-{month}-01", end=f"{year}-{month+1}-01", freq='M')[0] + pd.Timedelta(hours=21)
    date_range = pd.date_range(start, end, freq='3H')

    # Select the data at the specific timesteps
    dataset = dataset.sel(time=date_range)
    
    datasets.append(dataset)
    dataset.close()

In [62]:
test_2 = process_data(variable, year, level)

0 7 1st condition of 1990


In [7]:
year = int(year)
year_next = year + 1
month_act = [10, 11, 12]
month_next = [1, 2, 3]
if variable == 'geopotential':
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5_hourly_PL/'
else:
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

# Open and concatenate datasets
if year == 1990:
    dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
    dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
    dataset = xr.concat([dataset_act, dataset_next], dim='time')
    dataset = dataset.chunk({'time': 10})
elif year == 2021:
    dataset = open_and_concatenate(str(year), variable, month_next, way, level)
else:
    dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
    dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
    dataset = xr.concat([dataset_act, dataset_next], dim='time')
    dataset = dataset.chunk({'time': 10})

# Determine the specific variable to extract
specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time', 'level'])

# Import all tracks and convert dates
dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
dates['year'] = dates['start_date'].dt.year

# Find the indices for storms within the specified timeframe
if year == 1990:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
elif year == 2021:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
else:
# Chercher start_october dans year, sinon chercher dès janvier de year_next
    index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month >= 1))].index[0]
    index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
    #print(index_start_october, index_end_march_first, '3rd condition start_october + index_end_march_first')
    if len(index_end_march_first) > 0:
        index_end_march = index_end_march_first[-1]
        #print(index_end_march, 'index_end_march 1st condition of 2nd condition')
    else:
        # Si year_next ne renvoie rien, chercher la dernière instance de tempête dans year
        index_end_march = dates[((dates['end_date'].dt.year == year) & (dates['end_date'].dt.month <= 12))].index[-1]
        #print(index_end_march, 'index_end_march 2nd condition of 2nd condition')
# Process each storm
for i in range(index_start_october, index_end_march + 1):
    track = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/tc_irad_tracks/tc_3_hours/tc_irad_{i+1}.txt')
    start_date = dates.at[i, 'start_date']
    end_date = dates.at[i, 'end_date']
    storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

    # Initialize lists to store statistics
    stats = {'mean': [], 'min': [], 'max': [], 'std': []}
    #, 'skewness': [], 'kurtosis': []

    # Calculate statistics for each time step
    for time_step in storm_data.time:
        data_slice = storm_data.sel(time=time_step).values
        step_stats = calculate_statistics(data_slice)
        for key in stats:
            stats[key].append(step_stats[key])

    # Save statistics to CSV files
    for key in stats:
        pd.DataFrame(stats[key]).to_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets_3h/{variable}/storm_{i+1}/{key}_{i+1}_{level}.csv')

# Log the processing details
log_processing(variable, year, level, i+1)

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import sys
from datetime import datetime
from guppy import hpy

# Créer une instance de heapy
hp = hpy()
# might be a problem for ML data, since the data is quite large in size
# Define a function to open datasets and concatenate them
def open_and_concatenate(year, variable, months, way, level=0):
    datasets = [xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc') for month in months]
    if variable == 'geopotential' and level != 0:
        datasets = [dataset.sel(level=level) for dataset in datasets]
    return xr.concat(datasets, dim='time')

# Define a function to calculate statistics
def calculate_statistics(data_array):
    return {
        'mean': np.mean(data_array),
        'min': np.min(data_array),
        'max': np.max(data_array),
        'std': np.std(data_array),
    }

# Function to log processing details
def log_processing(variable, year, level, storm_number):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_message = f'Processed variable: {variable}, Year: {year}, Level: {level}, Timestamp: {timestamp}, Storm number:{storm_number}'
    with open(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets/processing_log.txt', 'a') as log_file:
        log_file.write(log_message + '\n')

# Créer une instance de heapy
hp = hpy()

# Main function to process data
def process_data(variable, year, level=0):
    year = int(year)
    year_next = year + 1
    month_act = [10, 11, 12]
    month_next = [1, 2, 3]
    if variable == 'geopotential':
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5_hourly_PL/'
    else:
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

    # Open and concatenate datasets
    if year == 1990:
        dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
        dataset = dataset.chunk({'time': 10})
    elif year == 2021:
        dataset = open_and_concatenate(str(year), variable, month_next, way, level)
    else:
        dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
        dataset = dataset.chunk({'time': 10})

    # Determine the specific variable to extract
    specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time', 'level'])

    # Import all tracks and convert dates
    dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
    dates['year'] = dates['start_date'].dt.year

    # Find the indices for storms within the specified timeframe
    if year == 1990:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
    elif year == 2021:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
    else:
    # Chercher start_october dans year, sinon chercher dès janvier de year_next
        index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month >= 1))].index[0]
        index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
        #print(index_start_october, index_end_march_first, '3rd condition start_october + index_end_march_first')
        if len(index_end_march_first) > 0:
            index_end_march = index_end_march_first[-1]
            #print(index_end_march, 'index_end_march 1st condition of 2nd condition')
        else:
            # Si year_next ne renvoie rien, chercher la dernière instance de tempête dans year
            index_end_march = dates[((dates['end_date'].dt.year == year) & (dates['end_date'].dt.month <= 12))].index[-1]
            #print(index_end_march, 'index_end_march 2nd condition of 2nd condition')
    # Process each storm
    for i in range(index_start_october, index_end_march + 1):
        track = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/tc_irad_tracks/tc_1_hour/tc_irad_{i+1}_interp.txt')
        start_date = dates.at[i, 'start_date']
        end_date = dates.at[i, 'end_date']
        storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

        # Initialize lists to store statistics
        stats = {'mean': [], 'min': [], 'max': [], 'std': []}
        #, 'skewness': [], 'kurtosis': []

        # Calculate statistics for each time step
        for time_step in storm_data.time:
            data_slice = storm_data.sel(time=time_step).values
            step_stats = calculate_statistics(data_slice)
            for key in stats:
                stats[key].append(step_stats[key])

        # Save statistics to CSV files
        for key in stats:
            pd.DataFrame(stats[key]).to_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets/{variable}/storm_{i+1}/{key}_{i+1}_{level}.csv')

    # Log the processing details
    log_processing(variable, year, level, i+1)

# Obtenir un instantané de l'utilisation de la mémoire
h = hp.heap()

# Imprimer l'information d'utilisation de la mémoire
print(h)

Partition of a set of 556294 objects. Total size = 73990366 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 157869  28 22672432  31  22672432  31 str
     1 129855  23  9451928  13  32124360  43 tuple
     2  38668   7  6867073   9  38991433  53 types.CodeType
     3  19704   4  5742824   8  44734257  60 dict (no owner)
     4  72160  13  5683431   8  50417688  68 bytes
     5  34732   6  4723552   6  55141240  75 function
     6   4408   1  4156480   6  59297720  80 type
     7   1605   0  2554920   3  61852640  84 dict of module
     8   4408   1  2273688   3  64126328  87 dict of type
     9   1655   0   905064   1  65031392  88 set
<1379 more rows. Type e.g. '_.more' to view.>


In [6]:
test_3 = process_data('10m_v_component_of_wind', 2020, 0)